In [1]:
import pandas as pd

In [2]:
all_features = ["tweet_type", "Language", "tweet_timestamp", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01",\
                    nrows = 100)
train['like_bool'] = train.like_timestamp.fillna(0)
train.loc[train.like_bool != 0.0, 'like_bool'] = 1.0
train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()
train = train[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]
train_set = train_set[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

val_set = val_set[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

In [3]:
def bool_to_str(value):
    "value should be a bool"
    return 'Yes' if value else 'No'

In [18]:
def construct_line(row, columns, dtypes, string_dict, f_train_set):
    global categorical_index
    print(categorical_index)
    numerical_index = 0
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        value_at_i = row[i]
        dtype_at_i = dtypes[i]
        column_name_at_i = columns[i]
        
        if ((dtype_at_i == object) or (dtype_at_i == bool)): # Categorical Features
            if (dtype_at_i == bool):
                value_at_i =  bool_to_str(value_at_i)
                
            values = string_dict[column_name_at_i]
            
            if value_at_i in [x[0] for x in values]:
                index = [x[0] for x in values].index(value_at_i)
                indexed_value = values[index][1]
                new_line = new_line + str(indexed_value)+":1 "
            else:
#                 indexed_value = sum(len(v) for v in string_dict.values()) + categorical_index
                indexed_value = categorical_index
                categorical_index = categorical_index + 1
                new_line = new_line + str(indexed_value)+":1 "
                values.append((value_at_i, indexed_value))
                string_dict[column_name_at_i] = values
        else: # Numerical Features
            new_line = new_line + str(numerical_index) + ":" +str(value_at_i)+" "
            numerical_index = numerical_index + 1
    print(new_line)
    f_train_set.write(new_line+"\n")
    return categorical_index

In [21]:
train_set

,tweet_type,Language,enaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,engaged_with_user_is_verified,engaging_user_id,enaging_user_follower_count,enaging_user_following_count,enaging_user_is_verified,engagee_follows_engager,like_bool
0,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,9586F04FF09A614BAE30367575053EC4,3418,4940,False,000043D9A730DF47697D0750F509B56A,871,927,False,True,1.0
1,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,9E1FD934C376903895DEEBCC0F6AB920,141370,3235,False,000006829BEADA9EEA695CF0C334B426,3,30,False,False,0.0
2,Retweet,D3164C7FBCF2565DDF915B1B3AEFB1DC,0437FEC24DCFCFF13F02E2AD94CA399E,7213153,540,True,00002C99ACC8931540C190542549BFE0,32,107,False,False,0.0
3,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,355C9DE0385BB017C038236942A81674,803025,133,True,00002C99ACC8931540C190542549BFE0,32,107,False,False,0.0
4,Retweet,9BF3403E0EB7EA8A256DA9019C0B0716,61B2BF22714D7AA1326F3905958DF402,119321,220,False,00002086C1D5C05ADE95E1C60FAF71FD,22,251,False,False,1.0
5,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1A2F2E91FEB94DBC54ADA40CC98FA557,2240268,4109,True,0000265D11F04218EACED1DB0B77C11F,31,137,False,False,0.0
6,TopLevel,06D61DCBBE938971E1EA0C38BD9B5446,0ACEEAEDC5CA2AEA34E3E37240A8319A,15716,892,False,000030DA986805A0B204966360B8AABB,143,119,False,True,0.0
7,Retweet,ECED8A16BE2A5E8871FD55F4842F16B1,46E6134373F19D742F24D6E72FC74360,440,412,False,00000860E80C67D8C46CE57C64DE9444,225,185,False,True,1.0
8,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,B481E206EC2F285C95524082FE1A7210,564546,10871,False,000019DCA89D7FD62B8D5DBD4C9C5FFA,220,1146,False,False,0.0
9,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,C3105DA40EC2F9A8382818F61FC238A0,214502,202,False,00002FAE5B0E6869DF7CBBB0D0CBF93A,8,88,False,False,1.0


In [20]:
cnames_numerical = list(train_set.select_dtypes(exclude=['object', 'bool']).columns)
categorical_index = len(cnames_numerical) # Categorical indices start from here.
cnames_categorical = list(train_set.select_dtypes(include=['object', 'bool']).columns)
string_dict = {k: [] for k in cnames_categorical}
f_train_set = open("train_set.csv", 'a')
train_set.apply(lambda x : construct_line(x, train_set.columns, train_set.dtypes, string_dict, f_train_set), axis = 1)
f_train_set.close()
f_val_set = open("val_set.csv", 'a')
categorical_index = val_set.apply(lambda x : construct_line(x, val_set.columns, val_set.dtypes, string_dict, f_val_set), axis = 1)
f_val_set.close()

5
1.0 5:1 6:1 7:1 0:3418 1:4940 8:1 9:1 2:871 3:927 10:1 11:1 
12
0.0 5:1 12:1 13:1 0:141370 1:3235 8:1 14:1 2:3 3:30 10:1 15:1 
16
0.0 16:1 6:1 17:1 0:7213153 1:540 18:1 19:1 2:32 3:107 10:1 15:1 
20
0.0 5:1 6:1 20:1 0:803025 1:133 18:1 19:1 2:32 3:107 10:1 15:1 
21
1.0 16:1 21:1 22:1 0:119321 1:220 8:1 23:1 2:22 3:251 10:1 15:1 
24
0.0 5:1 6:1 24:1 0:2240268 1:4109 18:1 25:1 2:31 3:137 10:1 15:1 
26
0.0 5:1 26:1 27:1 0:15716 1:892 8:1 28:1 2:143 3:119 10:1 11:1 
29
1.0 16:1 29:1 30:1 0:440 1:412 8:1 31:1 2:225 3:185 10:1 11:1 
32
0.0 5:1 6:1 32:1 0:564546 1:10871 8:1 33:1 2:220 3:1146 10:1 15:1 
34
1.0 5:1 6:1 34:1 0:214502 1:202 8:1 35:1 2:8 3:88 10:1 15:1 
36
1.0 5:1 21:1 36:1 0:357 1:329 8:1 37:1 2:673 3:961 10:1 11:1 
38
0.0 16:1 38:1 39:1 0:316272 1:303 18:1 40:1 2:29 3:188 10:1 15:1 
41
0.0 5:1 41:1 42:1 0:39786 1:32978 8:1 43:1 2:17 3:77 10:1 11:1 
44
1.0 5:1 29:1 44:1 0:3807 1:487 8:1 45:1 2:15 3:67 10:1 15:1 
46
0.0 5:1 29:1 46:1 0:517 1:369 8:1 47:1 2:220 3:278 10:1 11:1 
4

In [6]:
string_dict

{'tweet_type': [('TopLevel', 5), ('Retweet', 16), ('Quote', 50)],
 'Language': [('D3164C7FBCF2565DDF915B1B3AEFB1DC', 6),
  ('22C448FF81263D4BAF2A176145EE9EAD', 12),
  ('9BF3403E0EB7EA8A256DA9019C0B0716', 21),
  ('06D61DCBBE938971E1EA0C38BD9B5446', 26),
  ('ECED8A16BE2A5E8871FD55F4842F16B1', 29),
  ('975B38F44D65EE42A547283787FF5A21', 38),
  ('B9175601E87101A984A50F8A62A1C374', 41),
  ('4DC22C3F31C5C43721E6B5815A595ED6', 75),
  ('190BA7DA361BC06BC1D7E824C378064D', 85),
  ('167115458A0DBDFF7E9C0C53A83BAC9B', 114),
  ('125C57F4FA6D4E110983FB11B52EFD4E', 136)],
 'enaged_with_user_id': [('9586F04FF09A614BAE30367575053EC4', 7),
  ('9E1FD934C376903895DEEBCC0F6AB920', 13),
  ('0437FEC24DCFCFF13F02E2AD94CA399E', 17),
  ('355C9DE0385BB017C038236942A81674', 20),
  ('61B2BF22714D7AA1326F3905958DF402', 22),
  ('1A2F2E91FEB94DBC54ADA40CC98FA557', 24),
  ('0ACEEAEDC5CA2AEA34E3E37240A8319A', 27),
  ('46E6134373F19D742F24D6E72FC74360', 30),
  ('B481E206EC2F285C95524082FE1A7210', 32),
  ('C3105DA40EC2F9

### Read Test Set

In [7]:
all_features = ["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_timestamp"]

test = pd.read_csv("s3://recsys-challenge-2020/val.tsv", encoding="utf-8",
                    names = all_features, usecols= [6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 19, 23], sep="\x01", nrows = 10)
test['like_bool'] = test.like_timestamp.fillna(0)
test.loc[test.like_bool != 0.0, 'like_bool'] = 1.0
test = test[["tweet_type", "Language", "enaged_with_user_id",\
                "engaged_with_user_follower_count", "engaged_with_user_following_count", \
                "engaged_with_user_is_verified", "engaging_user_id", "enaging_user_follower_count",
                "enaging_user_following_count", "enaging_user_is_verified", "engagee_follows_engager", "like_bool"]]

### Write Train and Test Simultaneously

In [8]:
cnames_numerical = list(train.select_dtypes(exclude=['object', 'bool']).columns)
categorical_index = len(cnames_numerical)
cnames_categorical = list(train.select_dtypes(include=['object', 'bool']).columns)
string_dict = {k: [] for k in cnames_categorical}
f_train = open("train.csv", 'a')
train.apply(lambda x : construct_line(x, train.columns, train.dtypes, string_dict, categorical_index, f_train), axis = 1)
f_train.close()
f_test = open("test.csv", 'a')
test.apply(lambda x : construct_line(x, test.columns, test.dtypes, string_dict, categorical_index, f_test), axis = 1)
f_test.close()

0.0 5:1 6:1 7:1 0:3517 1:590 8:1 9:1 2:27428 3:600 10:1 11:1 
0.0 5:1 12:1 13:1 0:39786 1:32978 8:1 14:1 2:17 3:77 10:1 15:1 
1.0 5:1 6:1 16:1 0:141726 1:3232 8:1 17:1 2:2 3:29 10:1 11:1 
0.0 5:1 6:1 18:1 0:3044 1:99 8:1 17:1 2:3 3:30 10:1 11:1 
0.0 5:1 6:1 16:1 0:141370 1:3235 8:1 17:1 2:3 3:30 10:1 11:1 
0.0 19:1 6:1 20:1 0:3264 1:39 8:1 17:1 2:3 3:29 10:1 11:1 
0.0 5:1 6:1 21:1 0:18133 1:177 8:1 22:1 2:94 3:650 10:1 11:1 
1.0 5:1 23:1 24:1 0:320 1:319 8:1 25:1 2:225 3:185 10:1 15:1 
0.0 5:1 23:1 26:1 0:6099673 1:755 27:1 25:1 2:222 3:184 10:1 11:1 
1.0 19:1 23:1 28:1 0:440 1:412 8:1 25:1 2:225 3:185 10:1 15:1 
1.0 19:1 29:1 30:1 0:369 1:369 8:1 25:1 2:225 3:185 10:1 15:1 
0.0 19:1 31:1 32:1 0:1757 1:4999 8:1 33:1 2:216 3:391 10:1 15:1 
0.0 5:1 31:1 34:1 0:781 1:323 8:1 33:1 2:214 3:390 10:1 15:1 
1.0 19:1 31:1 35:1 0:128 1:493 8:1 33:1 2:218 3:394 10:1 15:1 
0.0 19:1 36:1 37:1 0:524 1:524 8:1 38:1 2:56 3:108 10:1 15:1 
1.0 19:1 6:1 39:1 0:1225 1:677 8:1 40:1 2:1139 3:46 10:1 15:1 
1

In [9]:
string_dict

{'tweet_type': [('TopLevel', 5), ('Retweet', 19), ('Quote', 43)],
 'Language': [('22C448FF81263D4BAF2A176145EE9EAD', 6),
  ('B9175601E87101A984A50F8A62A1C374', 12),
  ('ECED8A16BE2A5E8871FD55F4842F16B1', 23),
  ('D3164C7FBCF2565DDF915B1B3AEFB1DC', 29),
  ('06D61DCBBE938971E1EA0C38BD9B5446', 31),
  ('125C57F4FA6D4E110983FB11B52EFD4E', 36),
  ('190BA7DA361BC06BC1D7E824C378064D', 44),
  ('9BF3403E0EB7EA8A256DA9019C0B0716', 47),
  ('975B38F44D65EE42A547283787FF5A21', 63),
  ('167115458A0DBDFF7E9C0C53A83BAC9B', 85),
  ('4DC22C3F31C5C43721E6B5815A595ED6', 163)],
 'enaged_with_user_id': [('D7EFC6D19CFB09B9A1CE4C4B0E0091F6', 7),
  ('1F30A85E612A24B426013978ADB7AE46', 13),
  ('9E1FD934C376903895DEEBCC0F6AB920', 16),
  ('BEB3C337B266E2C4769065AD64724329', 18),
  ('E4CF13FFB0C7B8AC747476D57543C413', 20),
  ('D2D3B14476E8A5A33EC6435805B4B675', 21),
  ('6DCC8D7CDE39AF4BA022CFE8F1436656', 24),
  ('FE036351A95ABC3827A499067C340755', 26),
  ('46E6134373F19D742F24D6E72FC74360', 28),
  ('DA0AC09C87A3E0C

### Print Statements

In [ ]:
train_set.dtypes

In [ ]:
val_set.dtypes

In [ ]:
train_set.head()

In [ ]:
val_set.head()

In [ ]:
def construct_line_beta(row, columns, dtypes, string_dict, categorical_index):
    numerical_index = 0
    new_line = str(row[-1])+ " "
    for i in range(len(columns) - 1):
        if ((dtypes[i] == object) or (dtypes[i] == bool)):
            if (dtypes[i] == bool):
                row[i] =  bool_to_str(row[i])
            values = string_dict[columns[i]]
            if row[i] in [x[0] for x in values]:
                index = [x[0] for x in values].index(row[i])
                value_index = values[index][1]
                new_line = new_line + str(value_index)+":1 "
            else:
                value_index = sum(len(v) for v in string_dict.values()) + categorical_index
                new_line = new_line + str(value_index)+":1 "
                values.append((row[i], value_index))
                string_dict[columns[i]] = values
        else:
            new_line = new_line + str(numerical_index) + ":" +str(row[i])+" "
            numerical_index = numerical_index + 1
    print(new_line)